In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb

In [7]:
file_path = "../data/processed/adult_with_clusters.data"
df = pd.read_csv(file_path)
df.head()

,age,education-num,hours-per-week,capital-gain,capital-loss,workclass_Local-gov,workclass_Never-worked,workclass_Non-specific,workclass_Private,workclass_Self-emp-inc,...,occupation_Transport-moving,relationship_Other-relative,relationship_Own-child,relationship_Spouse,race_Other,race_White,sex_Male,native-country_United-States,income,cluster
0,39.0,13.0,40.0,2.843612,-0.221333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0
1,50.0,13.0,13.0,-0.298014,-0.221333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0,1
2,38.0,9.0,40.0,-0.298014,-0.221333,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0
3,53.0,7.0,40.0,-0.298014,-0.221333,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0,3
4,28.0,13.0,40.0,-0.298014,-0.221333,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,2


In [8]:
X = df.drop(columns=["income", "cluster"])
y = df["income"]
clusters = df["cluster"].unique()

In [9]:
models_and_params = {
    "RandomForest": (
        RandomForestClassifier(random_state=42, n_jobs=-1),
        {
            "n_estimators": [200, 500],
            "max_depth": [None, 20],             # shallow vs deep
            "max_features": ["sqrt", "log2"],
            "min_samples_split": [2, 10],        # regularization
        }
    ),

    "XGBoost": (
        xgb.XGBClassifier(
            use_label_encoder=False,
            eval_metric="logloss",
            random_state=42,
            tree_method="hist",
            n_jobs=-1
        ),
        {
            "n_estimators": [200, 500],
            "max_depth": [3, 6],
            "learning_rate": [0.1, 0.05],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0],
        }
    ),

    "SVM": (
        SVC(probability=True, random_state=42),
        {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"],
            "gamma": ["scale", "auto"]
        }
    )
}


In [10]:
for cluster in clusters:
    print("="*60)
    print(f"Cluster {cluster}")
    print("="*60)

    # Filter data for the cluster
    X_cluster = X[df["cluster"] == cluster]
    y_cluster = y[df["cluster"] == cluster]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_cluster, y_cluster, test_size=0.2, random_state=42, stratify=y_cluster
    )

    best_model = None
    best_score = 0
    best_name = None

    # Grid search for each model
    for name, (model, param_grid) in models_and_params.items():
        print(f"Running GridSearch for {name}...")
        grid = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
        grid.fit(X_train, y_train)

        y_pred = grid.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        print(f"{name} best params: {grid.best_params_}")
        print(f"{name} accuracy: {acc:.4f}")

        if acc > best_score:
            best_score = acc
            best_model = grid.best_estimator_
            best_name = name
            y_pred_best = y_pred

        print("-"*40)

    # Final report for the best model in this cluster
    print("\nBest model for this cluster:")
    print(f"Model: {best_name}")
    print(f"Accuracy: {best_score:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred_best))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_best))
    print("\n")

Cluster 0
Running GridSearch for RandomForest...
RandomForest best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 500}
RandomForest accuracy: 0.8214
----------------------------------------
Running GridSearch for XGBoost...


D:\UVG\NLP\text-normalization\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:43:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost best params: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'subsample': 1.0}
XGBoost accuracy: 0.8294
----------------------------------------
Running GridSearch for SVM...
SVM best params: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
SVM accuracy: 0.8048
----------------------------------------

Best model for this cluster:
Model: XGBoost
Accuracy: 0.8294
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88      1936
           1       0.78      0.68      0.72       953

    accuracy                           0.83      2889
   macro avg       0.81      0.79      0.80      2889
weighted avg       0.83      0.83      0.83      2889

Confusion Matrix:
[[1752  184]
 [ 309  644]]


Cluster 1
Running GridSearch for RandomForest...
RandomForest best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 500}
RandomForest accuracy: 0.8947
------

D:\UVG\NLP\text-normalization\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:50:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
XGBoost accuracy: 0.8971
----------------------------------------
Running GridSearch for SVM...
SVM best params: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
SVM accuracy: 0.8684
----------------------------------------

Best model for this cluster:
Model: XGBoost
Accuracy: 0.8971
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94       363
           1       0.80      0.29      0.43        55

    accuracy                           0.90       418
   macro avg       0.85      0.64      0.69       418
weighted avg       0.89      0.90      0.88       418

Confusion Matrix:
[[359   4]
 [ 39  16]]


Cluster 3
Running GridSearch for RandomForest...
RandomForest best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 200}
RandomForest accuracy: 0.8107
-----

...

XGBoost best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500, 'subsample': 1.0}
XGBoost accuracy: 0.9241
----------------------------------------
Running GridSearch for SVM...
SVM best params: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
SVM accuracy: 0.9074
----------------------------------------

Best model for this cluster:
Model: XGBoost
Accuracy: 0.9241
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96      2524
           1       0.75      0.41      0.53       295

    accuracy                           0.92      2819
   macro avg       0.84      0.70      0.75      2819
weighted avg       0.92      0.92      0.91      2819

Confusion Matrix:
[[2483   41]
 [ 173  122]]


Cluster 4
Running GridSearch for RandomForest...
RandomForest best params: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 200}
RandomForest accuracy: 0.7703
----

D:\UVG\NLP\text-normalization\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:56:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'subsample': 0.8}
XGBoost accuracy: 0.7780
----------------------------------------
Running GridSearch for SVM...
SVM best params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
SVM accuracy: 0.7392
----------------------------------------

Best model for this cluster:
Model: XGBoost
Accuracy: 0.7780
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       520
           1       0.78      0.66      0.71       381

    accuracy                           0.78       901
   macro avg       0.78      0.76      0.77       901
weighted avg       0.78      0.78      0.77       901

Confusion Matrix:
[[451  69]
 [131 250]]


Cluster 5
Running GridSearch for RandomForest...
RandomForest best params: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 200}
RandomForest accuracy: 0.9779
--

D:\UVG\NLP\text-normalization\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [22:58:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost best params: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
XGBoost accuracy: 0.9799
----------------------------------------
Running GridSearch for SVM...
SVM best params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
SVM accuracy: 0.9729
----------------------------------------

Best model for this cluster:
Model: XGBoost
Accuracy: 0.9799
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       967
           1       0.75      0.43      0.55        28

    accuracy                           0.98       995
   macro avg       0.87      0.71      0.77       995
weighted avg       0.98      0.98      0.98       995

Confusion Matrix:
[[963   4]
 [ 16  12]]


